In [1]:
import spatialdata as sd

import bento as bt
import spatialdata_plot

/home/ckmah/mambaforge/envs/spatialdata_refactor/lib/python3.10/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.11.3-CAPI-1.17.3) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/home/ckmah/mambaforge/envs/spatialdata_refactor/lib/python3.10/site-packages/spatialdata/__init__.py:11: UserWarning: Geopandas was set to use PyGEOS, changing to shapely 2.0 with:

	geopandas.options.use_pygeos = True

If you intended to use PyGEOS, set the option to False.
  _check_geopandas_using_shapely()


In [2]:
sdata = sd.read_zarr("/mnt/d/sdata/xenium_rep1_io/data.zarr")
sdata

SpatialData object with:
├── Images
│     ├── 'morphology_focus': MultiscaleSpatialImage[cyx] (1, 25778, 35416), (1, 12889, 17708), (1, 6444, 8854), (1, 3222, 4427), (1, 1611, 2213)
│     └── 'morphology_mip': MultiscaleSpatialImage[cyx] (1, 25778, 35416), (1, 12889, 17708), (1, 6444, 8854), (1, 3222, 4427), (1, 1611, 2213)
├── Points
│     └── 'transcripts': DataFrame with shape: (61276166, 8) (3D points)
├── Shapes
│     ├── 'cell_boundaries': GeoDataFrame shape: (167780, 1) (2D shapes)
│     ├── 'cell_circles': GeoDataFrame shape: (167780, 2) (2D shapes)
│     ├── 'nucleus_boundaries': GeoDataFrame shape: (167780, 1) (2D shapes)
│     └── 'xenium_landmarks': GeoDataFrame shape: (3, 2) (2D shapes)
└── Table
      └── AnnData object with n_obs × n_vars = 167780 × 313
    obs: 'cell_id', 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'region'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatialdata_attrs'

In [3]:
xyz_scale_vector = sd.transformations.get_transformation(
    sdata.points["transcripts"]
).to_scale_vector(["x", "y"])
xy_scale = sd.transformations.Scale(scale=xyz_scale_vector, axes=["x", "y"])
transform = sdata.points["transcripts"].attrs
transform["transform"]["global"] = xy_scale
sdata.points["transcripts"] = sd.models.PointsModel.parse(
    sdata.points["transcripts"].compute().reset_index(drop=True),
    coordinates={"x": "x", "y": "y"},
    feature_key="feature_name",
    instance_key="cell_id",
)
sdata.points["transcripts"].attrs = transform

INFO     Column `z` in `data` will be ignored since the data is 2D.                                                


In [4]:
csdata = sd.bounding_box_query(
    sdata,
    axes=["x", "y"],
    min_coordinate=[12000, 12000],
    max_coordinate=[12500, 12500],
    target_coordinate_system="global",
)

In [5]:
del csdata.images["morphology_focus"]
del csdata.images["morphology_mip"]

In [6]:
csdata.write(
    "/mnt/d/sdata/xenium_rep1_io/small_data.zarr",
    overwrite=True,
)